In [1]:
import pandas as pd
import os
import tensorflow as tf

C:\Users\lando\AppData\Local\Packages\PythonSoftwareFoundation.Python.3.10_qbz5n2kfra8p0\LocalCache\local-packages\Python310\site-packages\scipy\__init__.py:155: UserWarning: A NumPy version >=1.18.5 and <1.25.0 is required for this version of SciPy (detected version 1.26.2
  warnings.warn(f"A NumPy version >={np_minversion} and <{np_maxversion}"


In [2]:
base = "../../data/ml-data/nsq/"

dirs = [d for d in os.listdir(base) if d.endswith('.parquet')]

dfs = {}

for p in dirs:
    path = os.path.join(base, p)
    df_name = p.split('.')[0]
    dfs[df_name] = pd.read_parquet(path)

In [3]:
#print the length of every dataframe
for k in dfs:
    print(k, len(dfs[k]))

df_2022_01_1st 1231
df_2022_01_2nd 865
df_2022_02_1st 814
df_2022_02_2nd 405
df_2022_03_1st 462
df_2022_03_2nd 365
df_2022_04_1st 343
df_2022_04_2nd 552
df_2022_05_1st 286
df_2022_05_2nd 300
df_2022_06_1st 435
df_2022_06_2nd 310
df_2022_07_1st 311
df_2022_07_2nd 292
df_2022_08_1st 248
df_2022_08_2nd 382
df_2022_09_1st 309
df_2022_09_2nd 468
df_2022_10_1st 308
df_2022_10_2nd 524
df_2022_11_1st 383
df_2022_11_2nd 516
df_2022_12_1st 379
df_2022_12_2nd 864


In [9]:
i = 0
for df_name, df in dfs.items():
    print(df_name)
    first_row_body = df['body'][0]
    #show the first 5 rows of every dataframe in 'body'
    print(df['body'].head())
    print(first_row_body)
    
    
    i += 1
    if i == 1:
        break

df_2022_01_1st
0    There are two implicit parts to your question....
1    As someone who works in a pharmacy, it fucking...
2    I was diagnosed with cancer December 2019. Sur...
3    Oh my god. I'm Australian and I can't even fat...
4    “For Profit” has bled into EVERY SINGLE INDUST...
Name: body, dtype: object
There are two implicit parts to your question.  First, you're not distinguishing between the US, and each individual state in the US, and second, you're assuming that states can ban abortion but can't require vaccination.

First, each state has much greater power to regulate its citizens than the national government does.  States can require people to buy health insurance; the national government can't (and Obamacare had to be reinterpreted as a tax penalty that didn't force anyone to do anything).  States can ban alcohol; the national government can't.  The Supreme Court will be deciding whether any arm of the federal government can generally obligate vaccination, but it's u

In [ ]:
from transformers import pipeline, BartTokenizer

# Init
summarizer = pipeline("summarization", model="facebook/bart-large-cnn")
tokenizer = BartTokenizer.from_pretrained("facebook/bart-large-cnn")

max_tokens = 512  # Reduced max tokens

summaries = {}

rows_to_sum = 5

for df_name, df in dfs.items():
    print(df_name)
    if len(df) < rows_to_sum:
        summaries[df_name] = "Number of rows is less than i"
        continue

    for i in range(0, rows_to_sum):
        #get body i 
        body_i = df['body'].iloc[i] if not df['body'].empty else None
        
        # Check for empty text
        if not body_i:
            summaries[df_name] = "Empty body text"
            continue

        # Truncate the text more aggressively
        tokens = tokenizer(body_i, truncation=True, max_length=max_tokens, return_tensors="pt")["input_ids"]
        truncated_text = tokenizer.decode(tokens[0])

        #get the length of the truncated text
        truncated_text_length = len(truncated_text)
        #dynamically resize the summary based on length of the input body
        min_len = min(int(truncated_text_length * 0.25), 80)

        # Summarize the truncated text
        summary = summarizer(truncated_text, max_length=min_len+50, min_length=min_len, do_sample=False)[0]['summary_text']
        summaries[df_name] = summary

In [20]:
test1_body = next(iter(dfs))

print(test1_body)

df_2022_01_1st


In [21]:
test1 = next(iter(summaries))

print(dfs['df_2022_01_1st']['body'][0])

print("---Start Summary---")

print(f"{test1}: \n {summaries[test1]}")

There are two implicit parts to your question.  First, you're not distinguishing between the US, and each individual state in the US, and second, you're assuming that states can ban abortion but can't require vaccination.

First, each state has much greater power to regulate its citizens than the national government does.  States can require people to buy health insurance; the national government can't (and Obamacare had to be reinterpreted as a tax penalty that didn't force anyone to do anything).  States can ban alcohol; the national government can't.  The Supreme Court will be deciding whether any arm of the federal government can generally obligate vaccination, but it's undisputed that the state can (*see Jacobson v. Massachusetts*).

Second, *Roe v. Wade* says that the state has an interest in prenatal life that is balanced against the health and autonomy of the mother.  *Jacobson v. Massachusetts* says that the state has a compelling interest in vaccinating against contagious dis

In [23]:
#print the length of (dfs['df_2022_01_1st']['body'][0])
print(len(dfs['df_2022_01_1st']['body'][0].split()))

print(len(summaries[test1].split()))

200
87


In [35]:
dfs['df_2022_01_1st']

,author,author_cakeday,author_flair_css_class,author_flair_text,body,can_gild,controversiality,created_utc,distinguished,edited,...,permalink,retrieved_on,score,stickied,subreddit,subreddit_id,words,filtered_words,month,day
0,ThenaCykez,None,None,None,There are two implicit parts to your question....,True,0,2022-01-09 19:31:44,None,false,...,/r/NoStupidQuestions/comments/rzyuz0/serious_i...,2022-02-21 22:28:32,744,False,NoStupidQuestions,t5_2w844,"[there, are, two, implicit, parts, to, your, q...","[two, implicit, parts, question., , first,, di...",1,9
1,sanddem,None,None,None,"As someone who works in a pharmacy, it fucking...",True,0,2022-01-03 01:54:36,None,false,...,/r/NoStupidQuestions/comments/ruoss6/is_the_am...,2022-02-22 19:15:14,665,False,NoStupidQuestions,t5_2w844,"[as, someone, who, works, in, a, pharmacy,, it...","[someone, works, pharmacy,, fucking, sucks., p...",1,3
2,gracefulmunchkin,None,None,None,I was diagnosed with cancer December 2019. Sur...,True,0,2022-01-03 02:49:03,None,1.641189532E9,...,/r/NoStupidQuestions/comments/ruoss6/is_the_am...,2022-02-22 19:06:16,181,False,NoStupidQuestions,t5_2w844,"[i, was, diagnosed, with, cancer, december, 20...","[diagnosed, cancer, december, 2019., surgery, ...",1,3
3,Fatlantis,True,None,None,Oh my god. I'm Australian and I can't even fat...,True,0,2022-01-03 04:06:02,None,1.641183384E9,...,/r/NoStupidQuestions/comments/ruoss6/is_the_am...,2022-02-22 18:55:42,164,False,NoStupidQuestions,t5_2w844,"[oh, my, god., i'm, australian, and, i, can't,...","[oh, god., australian, even, fathom, this., la...",1,3
4,BooRadleysFriend,None,None,None,“For Profit” has bled into EVERY SINGLE INDUST...,True,0,2022-01-03 15:57:59,None,false,...,/r/NoStupidQuestions/comments/ruoss6/is_the_am...,2022-02-22 17:42:40,116,False,NoStupidQuestions,t5_2w844,"[“for, profit”, has, bled, into, every, single...","[“for, profit”, bled, every, single, industry....",1,3
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1226,LordSinguloth,None,None,None,For a while they were just getting unemploymen...,True,0,2022-01-10 13:31:27,None,false,...,/r/NoStupidQuestions/comments/s0gdbk/i_keep_he...,2022-02-21 20:12:07,-4,False,NoStupidQuestions,t5_2w844,"[for, a, while, they, were, just, getting, une...","[getting, unemployment, government., , , nowad...",1,10
1227,Sexual-Thunder69,None,None,None,There has never been a better time to be alive...,True,0,2022-01-12 00:46:14,None,false,...,/r/NoStupidQuestions/comments/s1sf24/do_you_th...,2022-02-21 15:23:04,-4,False,NoStupidQuestions,t5_2w844,"[there, has, never, been, a, better, time, to,...","[never, better, time, alive., , global, infant...",1,12
1228,Ravens1112003,None,None,None,Because they are still receiving pandemic rela...,True,1,2022-01-10 14:49:32,None,false,...,/r/NoStupidQuestions/comments/s0gdbk/i_keep_he...,2022-02-21 20:01:46,-5,False,NoStupidQuestions,t5_2w844,"[because, they, are, still, receiving, pandemi...","[still, receiving, pandemic, related, assistan...",1,10
1229,Jhadiro,None,None,None,"Nah, there are a HUGE number of people who are...",True,0,2022-01-13 15:26:52,None,false,...,/r/NoStupidQuestions/comments/s31qa9/is_joe_ro...,2022-02-21 10:52:05,-7,False,NoStupidQuestions,t5_2w844,"[nah,, there, are, a, huge, number, of, people...","[nah,, huge, number, people, vaccine, hesitant...",1,13


In [30]:
#print dfs size, type, and keys
print(len(dfs), type(dfs), dfs.keys())

24 <class 'dict'> dict_keys(['df_2022_01_1st', 'df_2022_01_2nd', 'df_2022_02_1st', 'df_2022_02_2nd', 'df_2022_03_1st', 'df_2022_03_2nd', 'df_2022_04_1st', 'df_2022_04_2nd', 'df_2022_05_1st', 'df_2022_05_2nd', 'df_2022_06_1st', 'df_2022_06_2nd', 'df_2022_07_1st', 'df_2022_07_2nd', 'df_2022_08_1st', 'df_2022_08_2nd', 'df_2022_09_1st', 'df_2022_09_2nd', 'df_2022_10_1st', 'df_2022_10_2nd', 'df_2022_11_1st', 'df_2022_11_2nd', 'df_2022_12_1st', 'df_2022_12_2nd'])


In [29]:
#show summaries size, type, and keys
print(summaries.keys())
print(type(summaries))
print(len(summaries))

dict_keys(['df_2022_01_1st', 'df_2022_01_2nd', 'df_2022_02_1st', 'df_2022_02_2nd', 'df_2022_03_1st', 'df_2022_03_2nd', 'df_2022_04_1st', 'df_2022_04_2nd', 'df_2022_05_1st', 'df_2022_05_2nd', 'df_2022_06_1st', 'df_2022_06_2nd', 'df_2022_07_1st', 'df_2022_07_2nd', 'df_2022_08_1st', 'df_2022_08_2nd', 'df_2022_09_1st', 'df_2022_09_2nd', 'df_2022_10_1st', 'df_2022_10_2nd', 'df_2022_11_1st', 'df_2022_11_2nd', 'df_2022_12_1st', 'df_2022_12_2nd'])
<class 'dict'>
24


Using Rogue to evaluate the summaries

In [26]:
from rouge import Rouge

rouge = Rouge()

rouge_scores = {}

for df_name in summaries:
    generated_summary = summaries[df_name]

    # Assuming each DataFrame in dfs has a column 'reference_summary' with the reference summaries
    reference_summary = dfs[df_name]['reference_summary'].iloc[0]  # Adjust this line as per your DataFrame structure

    # Calculate ROUGE scores
    scores = rouge.get_scores(generated_summary, reference_summary)
    
    # Store the scores
    rouge_scores[df_name] = scores

# Printing or processing the ROUGE scores
for df_name, scores in rouge_scores.items():
    print(f"ROUGE scores for {df_name}:")
    print(scores)

KeyError: 'reference_summary'